In [250]:
import numpy as np
import matplotlib.pyplot as plt

from typing import Callable, List, Tuple

N = 200
eps = 10**9

$$\int^1_0\frac {sin(\pi x^5)dx} {x^5 (1 - x)}$$
У интеграла две особенности
$$ \lim_{x \rightarrow 0} \frac {\sin(\pi x^5)} {x^5 (1 - x)} = 
    \pi \cdot \lim_{x \rightarrow 0} \frac {\sin(\pi x^5)} {\pi x^5} \cdot \frac {1}{ 1 - x } = \pi $$

$$ \lim_{x \rightarrow 1} \frac {\sin(\pi x^5)} {x^5 (1 - x)} = (0/0) =
    \pi \cdot \lim_{x \rightarrow 1} \frac {5x^4 \cdot \cos(\pi x^5)} {5x^4 - 6x^5} = 
    \pi \cdot \lim_{x \rightarrow 1} \frac {5\cos(\pi x ^ 5)} { 5 - 6x } = 5 \pi $$
    
Следовательно можно доопределить по непрерывности функцию в точке ноль и значение интеграла Римана не 
поменяется


In [251]:
def f(x : float) -> float:
    if (np.abs(x) < eps) :
        return np.pi 
    if (np.abs(x - 1) < eps):
        return 5 * np.pi
    return np.sin(np.pi * x ** 5) / ( x ** 5 * (1 - x))

$$\int_0^\infty e ^ { -\sqrt{x} + \sin(x/10) } dx = 
\int_0^1 \frac {e ^ { -\sqrt {\frac {t} {1 - t}} + \sin( \frac {t} {10(1 - t)})} } {(1 - t)^2}dt
$$

In [252]:
def g(t):
    x = t / (1 - t)
    return np.e ** (-np.sqrt(x) + np.sin(x/10))

In [253]:

def test(x : float) -> float: 
    return np.sin(x) 


In [254]:
#simpson formula
def integration(f : Callable[[float], float], interval : Tuple[float, float], n : int) -> float:
    a, b = interval
    h = (b - a) / n
    res = 0
    for i in range(1, n, 2):
        res += f(a + (i - 1) * h) + 4 * f(a + i * h) + f(a + (i + 1) * h)
    res *= h / 3
    return res 

$$ \int_{-1}^1 f(x) dx = \sum_{i = 1}^n w_i f(x_i)$$
где $w_i$ - вес квадратуры, $x_i$ - корни полинома лежандра

Метод Гаусса порядка 3
$$ 
x_0 = -\sqrt{\frac{3}{5}} \; w_0 = \frac {5}{9}\\
x_1 = 0 \; w_0 = \frac{8}{9} \\
x_2 = \sqrt{\frac{3}{5}} \; w_0 = \frac {5}{9}\\
$$

Как интегрировать функцию по произвольному промежутку $[a, b]$? Для этого воспользуемся следующей заменой
$$\int_a^bf(x)dx = \frac{b - a}{2} \cdot \int_{-1}^{1}f(\frac{b + a}{2} + \frac{b - a}{2} t)dt$$

Порядок сходимости простой формулы $$|R_n| \le \frac{n!}{(2n + 1)(2n!)^3 } M_{2n}(b - a)^{2n + 1}$$
Порядок сходимости составной формулы $O(h^{2n})$

In [255]:
# Gauss formula for n = 3

def gauss_integration_impl(f: Callable[[float], float]):
    # some static data
    roots = [-np.sqrt(3/5), 0, np.sqrt(3/5)]
    weight = [5/9, 8/9, 5/9]
    order = 3

    return sum(weight[i] * f(roots[i]) for i in range(order))

def gauss_integration(f: Callable[[float], float], interval : Tuple[float, float], n : int):
    a, b = interval
    h = (b - a) / n
    res = 0
    for i in range(0, n): 
        l, r = a + i * h, a + (i + 1) * h
        k1 = (r - l) / 2 
        k2 = (r + l) / 2
        res += k1 * gauss_integration_impl(lambda t : f(k2 + k1 * t))
    
    return res



теперь когда результат интегрирования неизвестен будем сравнивать I1 и I2 
$$ I_1 = I + C \cdot (R_1)^p $$
$$ I_2 = I + C \cdot (R_2)^p $$
$$ I_3 = I + C \cdot (R_3)^p $$

$$ I_2 - I_3 =  C \cdot (R_2 ^ p - R_3 ^ p) $$
$$ I_1 - I_3 = C \cdot (R_1 ^ p - R_3 ^ p) $$
$$ \frac {I_1 - I_3} {I_2 - I_3} = \frac {R_1 ^ p - R_3 ^ p} {R_2 ^ p - R_3 ^ p} $$

Неизвестные C, p, I\
magic happens: $$ 2^p $$




In [256]:
def calc_order(f: Callable[[float], float], interval : Tuple[float, float], n : int, method : Callable) -> float:
    I3 = method(test, interval, n//4)
    I2 = method(test, interval, n//2)
    I1 = method(test, interval, n)
    T1 = np.abs(I1 - I2)
    T2 = np.abs(I2 - I3)

    order = np.log2(T2/T1)
    return order

In [257]:
print("Порядок сходимости на тестовом интеграле: ")
print("Формула Симпсона", calc_order(test, interval=(0, np.pi), method=integration, n=1000))
print("Квадратура Гаусса при n=3", calc_order(test, interval=(0, np.pi), method=gauss_integration, n=50))

Порядок сходимости на тестовом интеграле: 
Формула Симпсона 3.999807630466989
Квадратура Гаусса при n=3 6.360797070956693


In [258]:
print("Порядок сходимости на первом интеграле: ")
print("Формула Симпсона", calc_order(f, interval=(0, 1), method=integration, n=1000))
print("Квадратура Гаусса при n=3", calc_order(f, interval=(0, 1), method=gauss_integration, n=50))
print("Значение интеграла: ", gauss_integration(f, (0, 1), 100))

Порядок сходимости на первом интеграле: 
Формула Симпсона 3.993871682508878
Квадратура Гаусса при n=3 6.159340250801146
Значение интеграла:  3.141592653589787


In [259]:
print("Порядок сходимости на втором интеграле: ")
print("Формула Симпсона", calc_order(g, interval=(0, 1), method=integration, n=1000))
print("Квадратура Гаусса при n=3", calc_order(g, interval=(0, 1), method=gauss_integration, n=50))
print("Значение интеграла: ", gauss_integration(g, (0, 1), 50))

Порядок сходимости на втором интеграле: 
Формула Симпсона 3.993871682508878
Квадратура Гаусса при n=3 6.159340250801146
Значение интеграла:  0.4125133054639926
